# Data exploration

Dit notebook zal worden gebruikt voor het vinden van interessante features.

In [13]:
# imported libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import TargetEncoder
warnings.simplefilter(action='ignore', category=FutureWarning)

In [14]:
# Kolommen die niet met n.v.t. of "?" werden aangegeven in de data dictionary.
# Zulke kolommen mochten wij negeren (volgens het interview)
cols_to_use= [
 '#stm_sap_meldnr',
 'stm_mon_nr',
 'stm_vl_post',
 'stm_sap_meld_ddt',
 'stm_sap_meldtekst_lang',
 'stm_sap_meldtekst',
 'stm_geo_mld',
 'stm_equipm_nr_mld',
 'stm_equipm_soort_mld',
 'stm_equipm_omschr_mld',
 'stm_km_van_mld',
 'stm_km_tot_mld',
 'stm_prioriteit',
 'stm_aanngeb_ddt',
 'stm_oh_pg_gst',
 'stm_geo_gst',
 'stm_equipm_nr_gst',
 'stm_equipm_soort_gst',
 'stm_equipm_omschr_gst',
 'stm_km_van_gst',
 'stm_km_tot_gst',
 'stm_oorz_groep',
 'stm_oorz_code',
 'stm_oorz_tkst',
 'stm_fh_ddt',
 'stm_fh_status',
 'stm_sap_storeind_ddt',
 'stm_tao_indicator',
 'stm_tao_indicator_vorige',
 'stm_tao_soort_mutatie',
 'stm_tao_telling_mutatie',
 'stm_tao_beinvloedbaar_indicator',
 'stm_sap_melddatum',
 'stm_sap_meldtijd',
 'stm_contractgeb_mld',
 'stm_techn_mld',
 'stm_contractgeb_gst',
 'stm_techn_gst',
 'stm_aanngeb_dd',
 'stm_aanngeb_tijd',
 'stm_aanntpl_dd',
 'stm_aanntpl_tijd',
 'stm_progfh_in_datum',
 'stm_progfh_in_tijd',
 'stm_progfh_in_invoer_dat',
 'stm_progfh_in_invoer_tijd',
 'stm_progfh_in_duur',
 'stm_progfh_gw_tijd',
 'stm_progfh_gw_lwd_datum',
 'stm_progfh_gw_lwd_tijd',
 'stm_progfh_gw_duur',
 'stm_progfh_gw_teller',
 'stm_fh_dd',
 'stm_fh_tijd',
 'stm_fh_duur',
 'stm_sap_storeinddatum',
 'stm_sap_storeindtijd',
 'stm_oorz_tekst_kort',
 'stm_pplg_van',
 'stm_pplg_naar',
 'stm_dstrglp_van',
 'stm_dstrglp_naar']

In [15]:
df = pd.read_csv("data/sap_storing_data_hu_project.csv", index_col=0, usecols=cols_to_use, engine='pyarrow')

In [16]:
df[['stm_sap_meld_ddt', 'stm_sap_meldtekst_lang','stm_geo_mld', 'stm_fh_duur', 'stm_progfh_in_duur', 'stm_oorz_tekst_kort']].sample(20)

,stm_sap_meld_ddt,stm_sap_meldtekst_lang,stm_geo_mld,stm_fh_duur,stm_progfh_in_duur,stm_oorz_tekst_kort
#stm_sap_meldnr,,,,,,
80605111,02/03/2011 15:50:00,Br-Hrt : Sp-HG km 3.0 knik in het spoor 'aanw'.,55,27.0,6,
80386085,04/12/2006 13:30:00,Aml wsl 41 uit controle (eenmalig),606,211.0,83,
80736442,27/09/2013 18:19:00,Akm : Thv Boornbrug zijn de borden tractie ...,9,3936.0,60,
80780963,27/09/2014 02:03:00,Zaanstraat : geen water sporen 29 en 30 waars...,917,176.0,90,
80409432,26/05/2007 20:30:00,Mt : Hacousto A1 gestoord,520,4035.0,45,
80745665,07/12/2013 12:59:00,Ht-Vga-Btl : Ovw-melder gestoord. Kontakt opn...,515,62.0,50,
80932768,27/03/2019 20:21:35,IVS10 Ech : Weerstation gestoord.,155,908.0,908,.
50089002,06/06/2009 11:45:00,AMFVA-BKDA: Verdwaalde man langs het spoor aa...,90.0,0.0,99999999,Verdwaalde man langs het spoor aangespro
80604395,25/02/2011 11:04:00,"2+Elst : achtergebleven tobs,in wsl sectie 406T",510,91.0,151,


## Target variabele aanmaken

Target variabele anm_tot_fh, welke de duur vanaf dat de aannemer ter plaatse is, tot het daadwerkelijke functieherstel. \
Dit is een van de betere keuzes voor de target variabele. \
Neem alle tijdstippen in de dataset: \
Melding komt binnen: Te vroeg, hier is nog maar weinig informatie. \
Aannemer gebeld: Gebeurt vlak nadat de melding binnenkomt. Dit is nog steeds vroeg. \
De aannemer is ten plaatse: Liever een latere prognose, wanneer de aannemer weet wat er aan de hand is. \
Prognose: Dit is de prognose van de aannemer. Interessant maar niet voor target. \
Invoer prognose: Dit zou het ideale punt zijn waarop wij een voorspelling geven. Dit vindt vaker (zie later in het notebook) laat in het proces, en rond functieherstel plaats. Vaak ook erna. \
Functieherstel: Dit is wat we willen weten. \
Einde storing: Dit is later dan wat we willen weten, dus niet relevant. \
Uitgebreide uitleg voor deze keuze is elders te vinden. \
**Conclusie: Wij nemen het tijdsverschil tussen aannemer aankomst en functieherstel**

In [17]:
# Convert columns to datetime type
df['stm_aanntpl_tijd'] = pd.to_datetime(df['stm_aanntpl_tijd'], format='%H:%M:%S', errors='coerce')
df['stm_aanntpl_dd'] = pd.to_datetime(df['stm_aanntpl_dd'], format='%d/%m/%Y', errors='coerce')
df['stm_fh_ddt'] = pd.to_datetime(df['stm_fh_ddt'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

# Combine date and time columns to a datetime column
df['stm_aanntpl_tijd'] = df['stm_aanntpl_tijd'].astype('str')
df['stm_aanntpl_dd'] = df['stm_aanntpl_dd'].astype('str')
df["aanntpl_ddt"] = df["stm_aanntpl_dd"] + " " + df["stm_aanntpl_tijd"].apply(lambda x: x.split(' ')[-1])
df['aanntpl_ddt'] = pd.to_datetime(df['aanntpl_ddt'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df = df.dropna(subset=['aanntpl_ddt'])

In [18]:
# Maak een kolom met de duur van de aannemer ter plaatse tot functieherstel
df['anm_tot_fh'] = df['stm_fh_ddt'] - df['aanntpl_ddt']
df['anm_tot_fh'] = df['anm_tot_fh'].apply(lambda x: x.seconds/60 + x.days * (24*60))

In [19]:
# Dit haalt zo'n 6000 rijen uit de database, tot 683985
df = df.dropna(subset=['anm_tot_fh'])

We halen negatieve waardes voor de targetvariabele uit de dataset. \
Deze zijn voor ons model, wat de functiehersteltijd wilt berekenen als de aannemer
ter plaatse is gekomen niet relevant. Het probleem is dan namelijk al opgelost. \
\
Verder is ons verteld dat wij storingen met een verwachte functieherstelduur van korter
dan 5 minuten of langer dan 8 uur mogen weglaten. \ 
Deze storingen zijn óf zodanig snel opgelost dat deze niet nuttig zijn om een voorspelling voor te doen, \
óf zodanig lang, dat het treinverkeer toch niet snel zal rijden, en het beter is om af te wachten.

In [20]:
# We halen negatieve waardes voor de targetvariabele uit de data
df = df[df['anm_tot_fh'] >= 0]
# We halen prognoses voor korter dan 5 minuten en langer dan 8 uur uit de data
df = df[(df['anm_tot_fh'] >= 5) & (df['anm_tot_fh'] <= 480)]

Door het aanmaken van de target variabele, NaN's en outliers eruit te halen, komen wij uit op 544583 resultaten.

### Duplicates
Zijn er duplicate in het model (mbt de index)

In [21]:
sum(df.index.duplicated())

194625

In [22]:
sum(df.duplicated())

41699

In [24]:
df = df[~df.index.duplicated(keep='first')]

Het verwijderen van rijen waarvan het #stm_sap_meldnr al eerder in de dataset is voorgekomen, reduceert de dataset tot 350.000.

### Functie voor het verwijderen van outliers

In [ ]:
def remove_outlier(df_in, col_name, k=3):
    """Removes rows in the dataframe which values in the given column are more than the given k (or 3) times
    the interquartile range removed from the 25 and/or 75 quantile."""
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3 - q1 
    fence_low  = q1 - k * iqr
    fence_high = q3 + k * iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

## Data exploration

#stm_sap_meldnr: De index. Niet een feature \
stm_mon_nr: Monitoringsnummer. Ziet er niet interessant uit. \
stm_vl_post:  \
stm_sap_meld_ddt:  \
stm_sap_meldtekst_lang:  \
stm_sap_meldtekst:  \
stm_geo_mld, stm_geo_gst: De geocode van de storing. Nader onderzoeken. \
stm_equipm_nr_mld \
stm_equipm_soort_mld \
stm_equipm_omschr_mld \
stm_km_van_mld, stm_km_van_gst \
stm_km_tot_mld, stm_km_tot_gst \
stm_prioriteit: Prioriteit van de storing. Interessante kolom. \
stm_aanngeb_ddt \
stm_oh_pg_gst \
stm_equipm_nr_gst \
stm_equipm_soort_gst \
stm_equipm_omschr_gst \
stm_oorz_groep: De oorzaakgroep. Wat is de oorzaak. Klinkt interessant. \
stm_oorz_code: De oorzaakcode. Wat is er mis. Klinkt veelbelovend. \
stm_oorz_tkst \
stm_fh_ddt \
stm_fh_status \
stm_sap_storeind_ddt \
stm_tao_indicator \
stm_tao_indicator_vorige \
stm_tao_soort_mutatie \
stm_tao_telling_mutatie \
stm_tao_beinvloedbaar_indicator \
stm_sap_melddatum \
stm_sap_meldtijd \
stm_contractgeb_mld, stm_contractgeb_gst: Contractgebied storing (aannemer). Klinkt interessant. \
stm_techn_mld, stm_techn_gst: Technieklabel storing. Klinkt interessant. \
stm_aanngeb_dd, stm_aanngeb_tijd: Datum/tijd aannemer gebeld. Buiten target. \
stm_aanntpl_dd, stm_aanntpl_tijd: Begin van duur target variabele. Geen feature. \
stm_progfh_in_datum, stm_progfh_in_tijd: Initiële datum/tijd prognose functieherstel. Zeer interessant. \
stm_progfh_in_invoer_dat, stm_progfh_in_invoer_tijd: Invoer tijd/datum initiële prognose? Interessant. \
stm_progfh_in_duur: Prognose initiële functieherstel. Zeer interessant. \
stm_progfh_gw_tijd, stm_progfh_gw_lwd_datum: Gewijzigde tijd/datum prognose \
stm_progfh_gw_lwd_tijd \
stm_progfh_gw_duur: Duur van de gewijzigde prognose, \
stm_progfh_gw_teller: Hoe vaak is de prognose van de aannemer gewijzigd. Enigzins interessant, moeilijk bruikbaar. \
stm_fh_dd, stm_fh_tijd: Datum/tijd functieherstel. Target valt hierbinnen. Niet interessant. \
stm_fh_duur: Duur vanaf melding tot functieherstel in minuten. Target is pas vanaf ter plaatse. Niet interessant. \
stm_sap_storeinddatum, stm_sap_storeindtijd: Datum/tijd einde storing. Dit is verder dan functieherstel, dus niet interessant. \
stm_oorz_tekst_kort \
stm_pplg_van \
stm_pplg_naar \
stm_dstrglp_van \
stm_dstrglp_naar \

### stm_progfh_in_duur

In [ ]:
# Remove nonsense values
dfprog = df
dfprog['stm_progfh_in_duur'] = df['stm_progfh_in_duur'].str.extract('(\d+)', expand=False)
dfprog['stm_progfh_in_duur'] = dfprog['stm_progfh_in_duur'].astype('int32')
dfprog = dfprog[dfprog.stm_progfh_in_duur != 99999999]
dfprog = dfprog[dfprog.stm_progfh_in_duur != 0]

In [ ]:
dfprog = remove_outlier(dfprog, 'stm_progfh_in_duur')

In [ ]:
plt.scatter(dfprog['stm_progfh_in_duur'], dfprog['anm_tot_fh'], s=0.3, alpha=0.1)
plt.title('Initiële prognose aannemer, functieherstelduur')
plt.xlabel('stm_progfh_in_duur')
plt.ylabel('anm_tot_fh')
# plt.plot(np.arange(0,500), np.repeat(60, 500))
# plt.plot(np.arange(0, 500), np.arange(0, 500), color='red')
plt.show()

In [ ]:
df.corr(numeric_only=True)['anm_tot_fh']

In [ ]:
# pd.options.display.max_rows = 1000
with pd.option_context("display.max_rows", 100):
    print(df['stm_oorz_code'].value_counts())

In [ ]:
# Convert columns to datetime type
df['stm_progfh_in_invoer_tijd'] = pd.to_datetime(df['stm_progfh_in_invoer_tijd'], format='%H:%M:%S', errors='coerce')
df['stm_progfh_in_invoer_dat'] = pd.to_datetime(df['stm_progfh_in_invoer_dat'], format='%d/%m/%Y', errors='coerce')
df['stm_fh_ddt'] = pd.to_datetime(df['stm_fh_ddt'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

# Combine date and time columns to a datetime column
df['stm_progfh_in_invoer_tijd'] = df['stm_progfh_in_invoer_tijd'].astype('str')
df['stm_progfh_in_invoer_dat'] = df['stm_progfh_in_invoer_dat'].astype('str')
df["stm_progfh_in_invoer_ddt"] = df["stm_progfh_in_invoer_dat"] + " " + df["stm_progfh_in_invoer_tijd"].apply(lambda x: x.split(' ')[-1])
df['stm_progfh_in_invoer_ddt'] = pd.to_datetime(df['stm_progfh_in_invoer_ddt'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df = df.dropna(subset=['stm_progfh_in_invoer_ddt'])

In [ ]:
df['duur_tot_prog'] = df['stm_progfh_in_invoer_ddt'] - df['aanntpl_ddt']
df['duur_tot_prog'] = df['duur_tot_prog'].apply(lambda x: x.seconds/60 + x.days * (24*60))

In [ ]:
df['prog_tov_fh_tijdstip'] = df['duur_tot_prog'] / df['anm_tot_fh']

In [ ]:
# plt.boxplot(df[(df.prog_tov_fh_tijdstip < 4) & (df.prog_tov_fh_tijdstip > 0)]['prog_tov_fh_tijdstip'])
plt.boxplot(df['prog_tov_fh_tijdstip'])

In [ ]:
df['stm_progfh_in_duur'].sample(20)

### stm_oorz_groep

In [ ]:
df['stm_oorz_groep'].unique()

In [ ]:
boxplot_data = [df[df.stm_oorz_groep == value]['anm_tot_fh'] for value in df['stm_oorz_groep'].unique()]

fig, ax = plt.subplots()
ax.boxplot(boxplot_data)
ax.set_xticklabels(df['stm_oorz_groep'].unique())
ax.set_ylabel('anm_tot_fh')
plt.show()

In [ ]:
for value in ['ONR-RIB', 'ONR-DERD', 'TECHONV', 'WEER']:
    print(df[df.stm_oorz_groep == value]['anm_tot_fh'].quantile([0.25, 0.5, 0.75]))

### stm_geo_mld

In [ ]:
median_time = []
for value in df['stm_geo_mld'].unique():
    median_time.append(df[(df.stm_geo_mld == value) & (df.stm_oorz_code == 225)]['anm_tot_fh'].median())

median_time = [x for x in median_time if str(x) != 'nan']

plt.boxplot(median_time)
plt.title('anm_tot_fh per geocode voor oorzaakcode 225')
plt.ylabel('Mediaan anm_tot_fh per geocode')

### stm_contractgeb_mld

In [ ]:
contract_time = []
for cont in df['stm_contractgeb_mld'].unique():
    contract_time.append(df[(df.stm_contractgeb_mld == cont) & (df.stm_oorz_code == 225)]['anm_tot_fh'].median())

contract_time = [x for x in contract_time if str(x) != 'nan']

plt.boxplot(contract_time)
plt.title('anm_tot_fh per contractgebied voor oorzaakcode 225')
plt.ylabel('Mediaan anm_tot_fh per contractgebied')
plt.show()
print('')

### stm_prioriteit

In [ ]:
boxplot_data = [df[df.stm_prioriteit == value]['anm_tot_fh'] for value in [1,2,4,5,8,9]]

fig, ax = plt.subplots()
ax.boxplot(boxplot_data)
ax.set_xticklabels([1,2,4,5,8,9])
ax.set_xlabel('Priority label')
ax.set_ylabel('anm_tot_fh')
plt.title('Boxplots for different priority values')
plt.show()

### stm_techn_mld

In [ ]:
boxplot_data = [df[df.stm_techn_mld == value]['anm_tot_fh'] for value in df['stm_techn_mld'].unique()]

fig, ax = plt.subplots()
ax.boxplot(boxplot_data)
ax.set_xticklabels(df['stm_techn_mld'].unique())
ax.set_xlabel('Techniekveld label')
ax.set_ylabel('Duur aannemer gearriveerd tot functieherstel')
plt.title('Boxplots for different techniekveld labels')
plt.show()

### stm_oorzaak_code

In [ ]:
len(df['stm_oorz_code'].unique())

In [ ]:
boxplot_data = [df[df.stm_oorz_code == value]['anm_tot_fh'] for value in df['stm_oorz_code'].unique()]

fig, ax = plt.subplots()
ax.boxplot(boxplot_data)
ax.set_xticklabels(df['stm_oorz_code'].unique())
ax.set_xlabel('Oorzaak code')
ax.set_ylabel('anm_tot_fh')
plt.title('Boxplots for different oorzaakcodes')
plt.show()